# Lecture 3: Pandas Filtering Mastery 🔍

Learn how to extract exactly the data you need from large datasets using pandas filtering techniques.

## What We'll Learn:
1. **Column Selection** - Pick specific columns from your DataFrame
2. **Row Filtering** - Select specific rows using index positions
3. **Boolean Filtering** - Use conditions to filter data
4. **Advanced Techniques** - Combine filters, use methods like `isin()`, `query()`
5. **Practice Questions** - Apply your knowledge!

Let's dive in with our cancer research dataset! 🧬

## Load Our Dataset

First, let's load our familiar DepMap cancer dataset.

In [ ]:
import pandas as pd
import numpy as np

# Load the DepMap CRISPR dataset
url = "https://zenodo.org/records/17098555/files/combined_model_crispr_data_filtered.csv?download=1"
df = pd.read_csv(url)

print(f"Dataset shape: {df.shape}")
print(f"\nFirst few columns: {list(df.columns[:10])}")
print(f"\nCancer types: {df['oncotree_lineage'].unique()[:10]}")

---
## 1. Column Selection 📋

### Single Column Selection
Select one column using bracket notation or dot notation.

In [ ]:
# Method 1: Bracket notation (recommended)
cell_lines = df['cell_line_name']
print("First 5 cell line names:")
print(cell_lines.head())
print(f"\nData type: {type(cell_lines)}")

# Method 2: Dot notation (only works if column name has no spaces/special characters)
model_ids = df.model_id
print(f"\nFirst 3 model IDs: {model_ids.head(3).tolist()}")

### Multiple Column Selection
Select multiple columns by passing a list of column names.

In [ ]:
# Select multiple columns - pass a list inside brackets
metadata_cols = ['model_id', 'cell_line_name', 'oncotree_lineage', 'oncotree_primary_disease']
metadata = df[metadata_cols]

print(f"Selected {len(metadata_cols)} columns:")
print(metadata.head())
print(f"\nShape: {metadata.shape}")

### Column Range Selection
Select a range of columns using slicing.

In [ ]:
# Select first 6 columns (all metadata)
metadata_all = df.iloc[:, :6]
print("All metadata columns:")
print(metadata_all.head())

# Select first 10 gene columns (columns 6-15)
first_genes = df.iloc[:, 6:16]
print(f"\nFirst 10 gene columns: {list(first_genes.columns)}")
print(first_genes.head(3))

---
## 2. Row Filtering by Position 📍

### Select Specific Rows
Use `.iloc[]` for position-based selection.

In [ ]:
# Select first 5 rows
first_five = df.iloc[:5]
print("First 5 rows (metadata only):")
print(first_five[['cell_line_name', 'oncotree_lineage']])

# Select last 3 rows
last_three = df.iloc[-3:]
print("\nLast 3 rows:")
print(last_three[['cell_line_name', 'oncotree_lineage']])

# Select specific row positions
specific_rows = df.iloc[[0, 10, 20, 30]]
print("\nRows at positions 0, 10, 20, 30:")
print(specific_rows[['cell_line_name', 'oncotree_lineage']])

### Sample Random Rows
Use `.sample()` to get random rows from your dataset.

In [ ]:
# Get 5 random rows
random_sample = df.sample(n=5, random_state=42)  # random_state for reproducibility
print("5 random cell lines:")
print(random_sample[['cell_line_name', 'oncotree_lineage', 'oncotree_primary_disease']])

# Get 10% of the dataset randomly
sample_10pct = df.sample(frac=0.1, random_state=42)
print(f"\n10% sample size: {len(sample_10pct)} rows")

---
## 3. Boolean Filtering 🎯

This is where pandas gets really powerful! Filter rows based on conditions.

### Basic Boolean Filtering

In [ ]:
# Filter for breast cancer cell lines
breast_filter = df['oncotree_lineage'] == 'Breast'
print("Boolean filter (first 10 values):")
print(breast_filter.head(10))
print(f"\nNumber of True values: {breast_filter.sum()}")

# Apply the filter
breast_cancer = df[breast_filter]
# Shorthand: breast_cancer = df[df['oncotree_lineage'] == 'Breast']

print(f"\nBreast cancer cell lines: {len(breast_cancer)}")
print(breast_cancer[['cell_line_name', 'oncotree_primary_disease']].head())

### Comparison Operators
Use different operators to create boolean conditions.

In [ ]:
# Let's look at gene A1BG scores
print("A1BG gene effect statistics:")
print(df['A1BG'].describe())

# Filter for cell lines where A1BG effect is very negative (< -0.2)
very_negative = df[df['A1BG'] < -0.2]
print(f"\nCell lines with A1BG effect < -0.2: {len(very_negative)}")
print(very_negative[['cell_line_name', 'oncotree_lineage', 'A1BG']].head())

# Filter for cell lines where A1BG effect is positive
positive_effect = df[df['A1BG'] > 0]
print(f"\nCell lines with positive A1BG effect: {len(positive_effect)}")

# Filter for cell lines with A1BG effect between -0.1 and 0.1
neutral_effect = df[(df['A1BG'] >= -0.1) & (df['A1BG'] <= 0.1)]
print(f"Cell lines with neutral A1BG effect (-0.1 to 0.1): {len(neutral_effect)}")

### Multiple Conditions with & and |
Combine multiple boolean conditions using `&` (and) and `|` (or).

In [ ]:
# Find breast cancer cell lines with very negative A1BG effect
breast_and_negative = df[(df['oncotree_lineage'] == 'Breast') & (df['A1BG'] < -0.1)]
print(f"Breast cancer cell lines with A1BG < -0.1: {len(breast_and_negative)}")
print(breast_and_negative[['cell_line_name', 'A1BG']].head())

# Find either breast OR lung cancer
breast_or_lung = df[(df['oncotree_lineage'] == 'Breast') | (df['oncotree_lineage'] == 'Lung')]
print(f"\nBreast OR Lung cancer cell lines: {len(breast_or_lung)}")
print(breast_or_lung['oncotree_lineage'].value_counts())

# Complex condition: Breast cancer with either very negative A1BG OR very positive A1BG
breast_extreme_a1bg = df[(df['oncotree_lineage'] == 'Breast') & 
                        ((df['A1BG'] < -0.15) | (df['A1BG'] > 0.15))]
print(f"\nBreast cancer with extreme A1BG effects: {len(breast_extreme_a1bg)}")
print(breast_extreme_a1bg[['cell_line_name', 'A1BG']].sort_values('A1BG'))

### String Filtering
Filter text columns using string methods.

In [ ]:
# Cell lines with 'MCF' in the name
mcf_lines = df[df['cell_line_name'].str.contains('MCF', na=False)]
print(f"Cell lines containing 'MCF': {len(mcf_lines)}")
print(mcf_lines[['cell_line_name', 'oncotree_lineage']])

# Cell lines starting with 'H'
h_lines = df[df['cell_line_name'].str.startswith('H')]
print(f"\nCell lines starting with 'H': {len(h_lines)}")
print(h_lines[['cell_line_name', 'oncotree_lineage']].head())

# Primary diseases containing 'Carcinoma'
carcinomas = df[df['oncotree_primary_disease'].str.contains('Carcinoma', na=False)]
print(f"\nCarcinoma cases: {len(carcinomas)}")
print(carcinomas['oncotree_primary_disease'].value_counts().head())

---
## 4. Advanced Filtering Techniques 🚀

### Using `.isin()` for Multiple Values

In [ ]:
# Filter for multiple cancer types at once
cancer_types_of_interest = ['Breast', 'Lung', 'Brain', 'Skin']
selected_cancers = df[df['oncotree_lineage'].isin(cancer_types_of_interest)]

print(f"Selected cancer types: {len(selected_cancers)} cell lines")
print(selected_cancers['oncotree_lineage'].value_counts())

# Filter for specific cell lines
important_lines = ['MCF7', 'HeLa', 'A549', 'HEK293T']
selected_lines = df[df['cell_line_name'].isin(important_lines)]
print(f"\nSelected cell lines found: {len(selected_lines)}")
if len(selected_lines) > 0:
    print(selected_lines[['cell_line_name', 'oncotree_lineage']])
else:
    print("None of the specified cell lines were found in this dataset")

### Using `.query()` Method
Write filters using a more readable syntax.

In [ ]:
# Query method - more readable for complex conditions
query_result = df.query("oncotree_lineage == 'Breast' and A1BG < -0.1")
print(f"Using .query(): {len(query_result)} breast cancer lines with A1BG < -0.1")

# Multiple conditions with query
complex_query = df.query("(oncotree_lineage == 'Breast' or oncotree_lineage == 'Lung') and A1BG > 0")
print(f"\nComplex query result: {len(complex_query)} lines")

# Query with variables
threshold = -0.05
cancer_type = 'Breast'
var_query = df.query(f"oncotree_lineage == '{cancer_type}' and A1BG < {threshold}")
print(f"\nUsing variables in query: {len(var_query)} lines")

### Filtering with `.loc[]`
Combine row and column selection in one operation.

In [ ]:
# Select specific rows AND columns
breast_metadata = df.loc[df['oncotree_lineage'] == 'Breast', 
                        ['cell_line_name', 'oncotree_primary_disease', 'A1BG', 'A1CF']]
print("Breast cancer metadata with first two genes:")
print(breast_metadata.head())

# Get specific gene values for specific conditions
high_a1bg_values = df.loc[df['A1BG'] > 0.1, 'A1BG']
print(f"\nCell lines with A1BG > 0.1: {len(high_a1bg_values)}")
print(f"Their A1BG values: {high_a1bg_values.tolist()}")

### Handling Missing Data in Filters

In [ ]:
# Check for missing values
print("Missing values per column (first 10):")
missing_counts = df.isnull().sum().head(10)
print(missing_counts[missing_counts > 0])  # Only show columns with missing values

# Filter out rows with missing values in specific column
no_missing_a1bg = df[df['A1BG'].notna()]
print(f"\nRows without missing A1BG: {len(no_missing_a1bg)}")

# Filter for rows with missing values
has_missing_a1bg = df[df['A1BG'].isna()]
print(f"Rows with missing A1BG: {len(has_missing_a1bg)}")

---
## 5. Combining Multiple Filters 🔗

Let's put it all together with some real research scenarios!

In [ ]:
# Research Question 1: Find breast cancer cell lines that are particularly sensitive 
# to multiple genes (A1BG and A1CF both negative)
sensitive_breast = df[(df['oncotree_lineage'] == 'Breast') & 
                     (df['A1BG'] < -0.05) & 
                     (df['A1CF'] < -0.05)]

print("Research Question 1: Breast cancer lines sensitive to both A1BG and A1CF")
print(f"Found {len(sensitive_breast)} cell lines:")
result1 = sensitive_breast[['cell_line_name', 'A1BG', 'A1CF']].round(3)
print(result1)

# Research Question 2: Compare gene effects across different cancer types
cancer_comparison = df[df['oncotree_lineage'].isin(['Breast', 'Lung', 'Brain'])]
comparison_data = cancer_comparison[['oncotree_lineage', 'A1BG', 'A1CF', 'A2M']]

print("\nResearch Question 2: Gene effects by cancer type")
gene_stats = comparison_data.groupby('oncotree_lineage').mean().round(3)
print(gene_stats)

---
## 📚 Practice Questions

Now it's your turn! Try to solve these filtering challenges using the cancer dataset.

### Question 1: Basic Filtering
Find all cell lines from 'Myeloid' cancer type. How many are there?

In [ ]:
# Your answer here:
# lung_cancer = ?
# print(f"Number of myleoid cancer cell lines: {len(myeloid_cancer)}")


### Question 2: Numerical Filtering
Find all cell lines where the A2M gene effect is greater than 0.1. Show their names and A2M values, sorted by A2M value (highest first).

In [ ]:
# Your answer here:
# high_a2m = ?
# result = ?
# print(result)


### Question 3: Multiple Conditions
Find breast cancer cell lines where BOTH A1BG < -0.1 AND A1CF > 0.05. How many cell lines meet both criteria?

In [ ]:
# Your answer here:
# complex_filter = ?
# print(f"Cell lines meeting both criteria: {len(complex_filter)}")
# if len(complex_filter) > 0:
#     print(complex_filter[['cell_line_name', 'A1BG', 'A1CF']])


### Question 4: String Filtering
Find all cell lines whose names contain the letter 'T'. What cancer types do they represent?

In [ ]:
# Your answer here:
# t_lines = ?
# print(f"Cell lines with 'T' in name: {len(t_lines)}")
# print("Cancer types:")
# print(?)


### Question 5: Advanced Challenge
Create a "drug target" dataset:
1. Filter for cancer types: 'Breast', 'Lung', 'Ovary'
2. Select only cell lines where A1BG effect is less than -0.1 (indicating the gene is essential)
3. Show only these columns: cell_line_name, oncotree_lineage, oncotree_primary_disease, A1BG
4. Sort by A1BG value (most negative first)
5. How many potential targets did you find?

In [ ]:
# Your answer here:
# Step 1: Filter cancer types
# target_cancers = ?

# Step 2: Filter for essential A1BG
# essential_a1bg = ?

# Step 3 & 4: Select columns and sort
# drug_targets = ?

# Step 5: Report results
# print(f"Potential A1BG targets found: {len(drug_targets)}")
# print(drug_targets)


---
## 🎯 Solutions

Try the questions above first, then run these cells to check your answers!

In [ ]:
# Solution 1
lung_cancer = df[df['oncotree_lineage'] == 'Lung']
print(f"Solution 1: Number of lung cancer cell lines: {len(lung_cancer)}")

In [ ]:
# Solution 2
high_a2m = df[df['A2M'] > 0.1]
result = high_a2m[['cell_line_name', 'A2M']].sort_values('A2M', ascending=False)
print(f"Solution 2: Cell lines with A2M > 0.1: {len(high_a2m)}")
print(result.head(10))  # Show top 10

In [ ]:
# Solution 3
complex_filter = df[(df['oncotree_lineage'] == 'Breast') & (df['A1BG'] < -0.1) & (df['A1CF'] > 0.05)]
print(f"Solution 3: Cell lines meeting both criteria: {len(complex_filter)}")
if len(complex_filter) > 0:
    print(complex_filter[['cell_line_name', 'A1BG', 'A1CF']].round(3))

In [ ]:
# Solution 4
t_lines = df[df['cell_line_name'].str.contains('T', na=False)]
print(f"Solution 4: Cell lines with 'T' in name: {len(t_lines)}")
print("Cancer types:")
print(t_lines['oncotree_lineage'].value_counts())

In [ ]:
# Solution 5
target_cancers = df[df['oncotree_lineage'].isin(['Breast', 'Lung', 'Ovary'])]
essential_a1bg = target_cancers[target_cancers['A1BG'] < -0.1]
drug_targets = essential_a1bg[['cell_line_name', 'oncotree_lineage', 'oncotree_primary_disease', 'A1BG']].sort_values('A1BG')

print(f"Solution 5: Potential A1BG targets found: {len(drug_targets)}")
print(drug_targets.round(3))

---
## 🎊 Congratulations!

You've mastered pandas filtering! Here's what you learned:

✅ **Column Selection**: Single columns, multiple columns, column ranges  
✅ **Row Filtering**: Position-based, random sampling  
✅ **Boolean Filtering**: Conditions, comparisons, combining filters  
✅ **Advanced Techniques**: `.isin()`, `.query()`, `.loc[]`  
✅ **Real Applications**: Filtering cancer research data  

**Next Steps**: 
- Practice with your own datasets
- Combine filtering with groupby operations
- Learn about joining/merging filtered datasets

Keep exploring! 🚀